In [ ]:
import os  # Importiere das Modul 'os' für die Arbeit mit dem Betriebssystem
import json  # Importiere das Modul 'json' für die Arbeit mit JSON-Daten
import torch  # Importiere das PyTorch-Framework
import torchvision.transforms as transforms  # Importiere Transformationen für Bilder in PyTorch
import torch.nn as nn  # Importiere die Klasse 'nn.Module' aus dem PyTorch-Modul 'torch.nn'
import matplotlib.pyplot as plt  # Importiere 'matplotlib.pyplot' zum Plotten von Diagrammen und Bildern
import matplotlib.patches as patches  # Importiere 'matplotlib.patches' zum Zeichnen von Rechtecken und Formen
import numpy as np  # Importiere das Modul 'numpy' für numerische Berechnungen

from sklearn.model_selection import train_test_split  # Importiere Funktionen zum Aufteilen von Daten in Trainings- und Testdaten
from torch.utils.data import Dataset  # Importiere die Klasse 'Dataset' aus 'torch.utils.data' für die Erstellung eines benutzerdefinierten Datasets
from torchvision.transforms import ToTensor  # Importiere die Transformation 'ToTensor' aus 'torchvision.transforms' für die Konvertierung von Bildern in Tensoren

from torchvision.models.detection import ssd  # Importiere das Modell 'ssd300_vgg16' aus 'torchvision.models.detection'
from PIL import Image  # Importiere die Klasse 'Image' aus dem Modul 'PIL' für die Arbeit mit Bildern

from tqdm import tqdm  # Importiere die Klasse 'tqdm' für die Fortschrittsanzeige


class DatasetLoader:
    def __init__(self, root_dir):
        self.root_dir = root_dir  # Das Stammverzeichnis des Datasets
        self.images_dir = os.path.join(root_dir, "images")  # Verzeichnis mit den Bildern
        self.annotations_dir = os.path.join(root_dir, "annotations")  # Verzeichnis mit den Annotationen
        
    def load_dataset(self):
        dataset = []  # Liste für das Dataset
        supported_image_extensions = (".jpg", ".jpeg", ".png")  # Unterstützte Bildformate
        supported_annotation_extensions = (".jpg.json", ".jpeg.json", ".png.json")  # Unterstützte Annotationen (JSON-Dateien)
        
        for filename in os.listdir(self.images_dir):  # Schleife über alle Dateien im Verzeichnis mit den Bildern
            if filename.lower().endswith(supported_image_extensions):  # Prüfe, ob die Datei eine unterstützte Bildendung hat
                image_path = os.path.join(self.images_dir, filename)  # Pfad zur Bilddatei
                
                # Annotationen
                annotation_filename = os.path.splitext(filename)[0]  # Dateiname ohne Erweiterung
                for extension in supported_annotation_extensions:  # Suche nach einer unterstützten Annotation für das Bild
                    annotation_file = annotation_filename + extension
                    annotation_path = os.path.join(self.annotations_dir, annotation_file)
                    if os.path.exists(annotation_path):
                        break  # Breche die Schleife ab, sobald eine passende Annotation gefunden wurde

                # Lade das Bild und die Annotationen
                image, annotations = self._read_data(image_path, annotation_path)
                dataset.append((image, annotations))  # Füge das Bild und die Annotationen zum Dataset hinzu

        return dataset  # Gib das geladene Dataset zurück

    def _read_data(self, image_path, annotation_path):
        image = Image.open(image_path)  # Öffne das Bild mit PIL
        
        with open(annotation_path, 'r') as f:  # Öffne die Annotationen als JSON-Datei
            annotations = json.load(f)  # Lade die Annotationen aus der JSON-Datei
        
        image_annotations = {  # Erstelle ein Dictionary für Bild und Annotationen
            "filename": annotations["FileName"],  # Dateiname des Bildes
            "annotations": []  # Liste für die Annotationen des Bildes
        }
        
        for annotation in annotations["Annotations"]:  # Schleife über alle Annotationen
            bbox = annotation["BoundingBox"]  # Bounding-Box-Koordinaten
            xmin, ymin, xmax, ymax = bbox  # Koordinaten der Bounding Box
            label = annotation["classname"]  # Label der Annotation
            
            formatted_annotation = {
                "bbox": [xmin, ymin, xmax, ymax],  # Bounding-Box-Koordinaten als Liste
                "label": label  # Label der Annotation
            }
            
            image_annotations["annotations"].append(formatted_annotation)  # Füge die formatierte Annotation zum Bild hinzu
        
        return image, image_annotations  # Gib das Bild und die Annotationen zurück
    
    
    
root_dir = r"C:\Users\Domi\Documents\GitHub\Deep-Vision-sta\Datasets\Face Mask Detection Dataset\Medical mask\Medical mask\Medical Mask"  # Wurzelverzeichnis des Datasets

dataset_loader = DatasetLoader(root_dir)  # Erstelle einen DatasetLoader mit dem Wurzelverzeichnis
dataset = dataset_loader.load_dataset()  # Lade das Dataset
image, annotations = dataset[0]  # Nehme das erste Bild und die zugehörigen Annotationen
print(len(dataset))  # Gib die Anzahl der geladenen Bilder im Dataset aus




class MyCustomDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset  # Das zugrunde liegende Dataset
        self.transform = ToTensor()  # Transformation zum Konvertieren des Bildes in einen Tensor

    def __len__(self):
        return len(self.dataset)  # Gib die Länge des zugrunde liegenden Datasets zurück

    def __getitem__(self, index):
        data = self.dataset[index]  # Hole das Datenobjekt aus dem zugrunde liegenden Dataset
        image = self.transform(data[0])  # Wende die Transformation auf das Bild an, um es in einen Tensor umzuwandeln
        annotations = data[1]  # Behalte die Annotationen bei

        return image, annotations  # Gib das Bild und die Annotationen zurück
    


def collate_fn(batch):
    images = []  # Liste für die Bilder
    annotations = []  # Liste für die Annotationen
    annotation_list = []  # Separate Liste für die Annotationen

    target_size = (300, 300)  # Zielgröße für das Rescaling

    rescale_transform = transforms.Resize(target_size, interpolation=Image.Resampling.BILINEAR)  # Rescaling-Transformation

    for image, annotation in batch:
        image_size = image.size()  # Größe des Bildes
        image = transforms.ToPILImage()(image)  # Konvertiere den Tensor in eine PIL-Image-Instanz

        image = rescale_transform(image)  # Wende die Rescaling-Transformation auf das Bild an

        image = transforms.ToTensor()(image) images.append(image)  # Füge das Bild zur Liste der Bilder hinzu

        width_ratio = target_size[0] / image_size[2]  # Verhältnis der Breiten
        height_ratio = target_size[1] / image_size[1]  # Verhältnis der Höhen

        for bbox_dict in annotation['annotations']:
            bbox = bbox_dict['bbox']
            x_min, y_min, x_max, y_max = bbox
            x_min *= width_ratio  # Skaliere die Bounding-Box-Koordinaten entsprechend dem Verhältnis der Breiten
            y_min *= height_ratio  # Skaliere die Bounding-Box-Koordinaten entsprechend dem Verhältnis der Höhen
            x_max *= width_ratio  # Skaliere die Bounding-Box-Koordinaten entsprechend dem Verhältnis der Breiten
            y_max *= height_ratio  # Skaliere die Bounding-Box-Koordinaten entsprechend dem Verhältnis der Höhen
            bbox_dict['bbox'] = [x_min, y_min, x_max, y_max]  # Aktualisiere die Bounding-Box-Koordinaten in der Annotation

        annotations.append(annotation)  # Füge die Annotation zur Liste der Annotationen hinzu

    images = torch.stack(images)  # Staple die Bilder zu einem Tensor

    return images, annotations  # Gib die Bilder und Annotationen zurück




model = ssd.ssd300_vgg16(num_classes=20)  # Initialisiere das Modell 'ssd300_vgg16' mit 20 Klassen

train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)  # Teile das Dataset in Trainings- und Testdaten auf

train_dataset = MyCustomDataset(train_data)  # Erstelle ein benutzerdefiniertes Trainingsdataset
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)  # Erstelle den Trainingsdataloader mit Stapelung
#train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)

test_dataset = MyCustomDataset(test_data)  # Erstelle ein benutzerdefiniertes Testdataset
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=False, collate_fn=collate_fn)  # Erstelle den Testdataloader mit Stapelung

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Definiere den Optimizer mit SGD und Lernrate 0.001 und Momentum 0.9
criterion = nn.CrossEntropyLoss()  # Definiere die Loss-Funktion als Cross-Entropy-Loss



def visualize_sample(dataloader, x):
    images, annotations = next(iter(dataloader))  # Hole das x-te Element aus dem Dataloader
    image = images[x]  # Wähle das x-te Bild
    boxes = annotations[x]['annotations']  # Hole die Annotationen für das x-te Bild

    fig, ax = plt.subplots(1)  # Erstelle eine neue Figur und Achse

    ax.imshow(image.permute(1, 2, 0))  # Zeige das Bild in der Achse an

    for box in boxes:  # Iteriere über die Bounding Boxes und zeichne Rechtecke in der Achse
        x_min, y_min, x_max, y_max = box['bbox']
        width = x_max - x_min
        height = y_max - y_min
        rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)

    plt.show()  # Zeige die visualisierten Bounding Boxes an

visualize_sample(train_dataloader, 4)  # Visualisiere das 4. Beispiel im Trainingsdataloader



def draw_image_with_boxes(image, target):
    image = image.cpu().permute(1, 2, 0).numpy()  # Konvertiere das Bild in ein NumPy-Array

    boxes = target["boxes"]  # Bounding-Box-Koordinaten
    labels = target["labels"]  # Labels
    boxes = boxes.cpu().numpy()  # Konvertiere die Bounding-Box-Koordinaten in ein NumPy-Array
    labels = labels.cpu().numpy()  # Konvertiere die Labels in ein NumPy-Array
        
    fig, ax = plt.subplots(1)  # Erstelle eine neue Figur und Achse
    
    ax.imshow(image)  # Zeige das Bild in der Achse an
    
    for box, label in zip(boxes, labels):  # Iteriere über die Bounding Boxes und zeichne Rechtecke in der Achse
        x_min, y_min, x_max, y_max = box
        width = x_max - x_min
        height = y_max - y_min
        rect = patches.Rectangle((x_min, y_min), width, height, linewidth=2, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x_min, y_min, f"Label: {label}", color='r', fontsize=8, bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

    plt.show()  # Zeige die Achse

# Trainingsschleife
num_epochs = 10  # Anzahl der Epochen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Verwende die GPU, falls verfügbar, ansonsten die CPU
model.to(device)  # Verschiebe das Modell auf das entsprechende Gerät (CPU oder GPU)
model.train()  # Setze das Modell in den Trainingsmodus

for epoch in range(num_epochs):  # Schleife über die Epochen
    pbar = tqdm(train_dataloader, total=len(train_dataloader))  # Fortschrittsanzeige über den Trainingsdataloader
    total_loss = 0.0  # Gesamtverlust pro Epoche
    
    for images, annotations in pbar:  # Schleife über die Batches im Trainingsdataloader
        images = images.to(device)  # Verschiebe die Bilder auf das entsprechende Gerät (CPU oder GPU)

        targets = []  # Liste für die Annotationen

        class_mapping = {  # Mapping der Klassenlabels
            "hijab_niqab": 0,
            "mask_colorful": 1,
            "mask_surgical": 2,
            "face_no_mask": 3,
            "face_with_mask_incorrect": 4,
            "face_with_mask": 5,
            "face_other_covering": 6,
            "scarf_bandana": 7,
            "balaclava_ski_mask": 8,
            "face_shield": 9,
            "other": 10,
            "gas_mask": 11,
            "turban": 12,
            "helmet": 13,
            "sunglasses": 14,
            "eyeglasses": 15,
            "hair_net": 16,
            "hat": 17,
            "goggles": 18,
            "hood": 19
        }  # Mapping der Klassen zu numerischen Werten

        for annotation in annotations:  # Schleife über die Annotationen
            boxes = annotation["annotations"]  # Bounding-Box-Koordinaten
            labels = [box["label"] for box in boxes]  # Labels der Bounding Boxes
            bboxes = [box["bbox"] for box in boxes]  # Bounding-Box-Koordinaten

            labels = [class_mapping[label] for label in labels]  # Konvertiere die Labels in numerische Werte

            target = {
                "boxes": torch.tensor(bboxes, dtype=torch.float32).to(device),  # Konvertiere die Bounding-Box-Koordinaten in Tensoren und verschiebe sie auf das entsprechende Gerät
                "labels": torch.tensor(labels).to(device)  # Konvertiere die Labels in Tensoren und verschiebe sie auf das entsprechende Gerät
            }
            targets.append(target)  # Füge die Annotationen zur Liste der Annotationen hinzu

        optimizer.zero_grad()  # Setze die Gradienten zurück

        loss_dict = model(images, targets)  # Berechne den Verlust
        losses = sum(loss for loss in loss_dict.values())  # Summiere die Verluste
        losses.backward()  # Backpropagation
        optimizer.step()  # Aktualisiere die Gewichte des Modells

        total_loss += losses.item()  # Addiere den Verlust zum Gesamtverlust
        average_loss = total_loss / (pbar.n + 1)  # Berechne den durchschnittlichen Verlust pro Batch

        pbar.set_description(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")  # Aktualisiere die Fortschrittsanzeige

    pbar.close()  # Schließe die Fortschrittsanzeige
